In [1]:

import requests
from config import geoapify_key
from config2 import geoapify_key as geoapify_key2
import csv
import pandas as pd
import pprint
import json
import time 



In [2]:
# get the data from the json file
json_path = "./Resources/IMDB_API_100_location.json"
# json_path = "./Resources/IMDB_API_100_location.json"
df = pd.read_json(json_path)
# remove the duplicate rows by location_name
df = df.drop_duplicates(subset=['location_name'], keep='first')

# create a new list to store the data
location_list = []
df.head(10)



,id,location_id,location_name
0,tt11858890,/title/tt11858890/filminglocations/lc2749017,"Bangkok, Thailand"
1,tt11858890,/title/tt11858890/filminglocations/lc2763749,"Pinewood Studios, Pinewood Road, Iver Heath, I..."
2,tt11858890,/title/tt11858890/filminglocations/lc2749018,Thailand
3,tt11858890,/title/tt11858890/filminglocations/lc3578189,"Siem Reap, Cambodia"
4,tt15009428,/title/tt15009428/filminglocations/lc3375865,"London, England, UK"
5,tt15009428,/title/tt15009428/filminglocations/lc3375866,Greece
6,tt17024450,/title/tt17024450/filminglocations/lc2973962,"Minori, Campania, Italy"
7,tt17024450,/title/tt17024450/filminglocations/lc2973960,"Atrani, Campania, Italy"
8,tt17024450,/title/tt17024450/filminglocations/lc3493652,"Maiori, Campania, Italy"
9,tt17024450,/title/tt17024450/filminglocations/lc2973961,"Ravello, Campania, Italy"


In [3]:
# loop through the rows and get the location data


def findthelocation(name):
    locationinfo = {}
    locationinfo["location_name"] = name
    name = name.replace("UK","United Kingdom")
    name = name.replace("USA","United States of America")

    
    url = "https://api.geoapify.com/v1/geocode/autocomplete?text="+name+"&format=json&apiKey="+geoapify_key2
    
    response = requests.get(url)
    
    if 'results' in response.json():
        if len(response.json()['results'])==0:
    
            if "," in name:
                name = name.split(",",1)[1]
                return findthelocation(name)
             
            else:
                locationinfo["lat"] = "N/A"
                locationinfo["lon"] = "N/A"
          
                return locationinfo
        
        else:
            locationinfo["lat"] = response.json()["results"][0]["lat"]
            locationinfo["lon"] = response.json()["results"][0]["lon"]

        
        return locationinfo
    else:
        return findthelocation(name.split(",",1)[1])




In [4]:
def check_value_in_list_of_dicts(list_of_dicts, value):
    
    for dictionary in list_of_dicts:

        if value in dictionary.get('location_name'):
            return True
    return False

In [5]:
i=0
for index, row in df.iterrows():
    i=i+1
    print(i)
    
# for index, row in df.iterrows():
    # time.sleep(0.5)
    text = row["location_name"]
    print(text)
    with open(json_path, 'r') as file:
        data = json.load(file)
        if not isinstance(data, list):
            data = [data]
            data = [item for item in data if 'lon' in item]
        if check_value_in_list_of_dicts(data,text):
            print('We have it')   
        else:
            time.sleep(0.5)
            location=findthelocation(text)    
            print(text)
            pprint.pprint(location)
            with open(json_path, 'w') as file:
                
                data.append(location)
                json.dump(data, file, indent=4)
                print("add a row")
            
    # location_list.append(location)





1
Bangkok, Thailand
We have it
2
Pinewood Studios, Pinewood Road, Iver Heath, Iver, Buckinghamshire, England, UK
We have it
3
Thailand
We have it
4
Siem Reap, Cambodia
We have it
5
London, England, UK
We have it
6
Greece
We have it
7
Minori, Campania, Italy
We have it
8
Atrani, Campania, Italy
We have it
9
Maiori, Campania, Italy
We have it
10
Ravello, Campania, Italy
We have it
11
Cinecittà Studios, Cinecittà, Rome, Lazio, Italy
We have it
12
Napoli, Campania, Italy
We have it
13
Italy
We have it
14
Phuket, Thailand
We have it
15
Los Angeles, California, USA
We have it
16
Hong Kong, China
We have it
17
Warner Brothers Burbank Studios - 4000 Warner Boulevard, Burbank, California, USA
We have it
18
China
We have it
19
Helsetkopen, Møre og Romsdal, Norway
We have it
20
Hellesylt, Norway
We have it
21
Stoney Middleton, Derbyshire, England, UK
We have it
22
Grand Central, Birmingham, West Midlands, England, UK
We have it
23
Rome, Lazio, Italy
We have it
24
Åndalsnes, Rauma, Møre og Romsdal

In [6]:
# # create a new dataframe
# location_df = pd.DataFrame(location_list)
# location_df.head()
# # reorder the columns
# location_df = location_df[["location_name","lat","lon"]]
# # save the dataframe to csv file
# location_df.to_csv("./Resources/IMDB_API_100_location_geoapify_test.csv", index=False)
# location_df.to_json("./Resources/IMDB_API_100_location_geoapify_test.json", orient="records")
